# Ajustar um modelo de IA generativo para resumo de diálogo

Neste caderno, você ajustará um LLM existente do Hugging Face para um resumo aprimorado do diálogo. Você usará o modelo [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5), que fornece um modelo ajustado para instruções de alta qualidade e pode resumir o texto imediatamente. Para melhorar as inferências, você explorará uma abordagem completa de ajuste fino e avaliará os resultados com métricas ROUGE. Em seguida, você realizará o ajuste fino eficiente de parâmetros (PEFT), avaliará o modelo resultante e verá que os benefícios do PEFT superam as métricas de desempenho ligeiramente inferiores.

# índice

- [1 - Configurar Kernel, Carregar Dependências Necessárias, Conjunto de Dados e LLM](#1)
   - [1.1 - Configurar Kernel e Dependências Necessárias](#1.1)
   - [1.2 - Carregar conjunto de dados e LLM](#1.2)
   - [ 1.3 - Teste o modelo com inferência de disparo zero](#1.3)
- [2 - Execute o ajuste fino completo](#2)
   - [2.1 - Pré-processar o conjunto de dados de resumo de diálogo](#2.1)
   - [2.2 - Ajustar o modelo com o conjunto de dados pré-processado](#2.2)
   - [ 2.3 - Avaliar o modelo qualitativamente (avaliação humana)](#2.3)
   - [2.4 - Avaliar o modelo quantitativamente (com métrica ROUGE)](#2.4)
- [3 - Executar ajuste fino eficiente de parâmetros (PEFT)](#3)
   - [3.1 - Configure o modelo PEFT/LoRA para ajuste fino](#3.1)
   - [3.2 - Adaptador PEFT de trem](#3.2)
   - [ 3.3 - Avaliar o Modelo Qualitativamente (Avaliação Humana)](#3.3)
   - [3.4 - Avaliar o modelo quantitativamente (com métrica ROUGE)](#3.4)

<a nome='1'></a>
## 1 - Configurar Kernel, Carregar Dependências Necessárias, Conjunto de Dados e LLM

<a nome='1.1'></a>
### 1.1 - Configurar o kernel e as dependências necessárias

In [1]:
!pip install torch torchdata

!pip install transformers datasets evaluate rouge_score loralib peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

<a nome='1.2'></a>
### 1.2 - Carregar conjunto de dados e LLM

Você continuará experimentando o conjunto de dados [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) Hugging Face. Ele contém mais de 10.000 diálogos com resumos e tópicos rotulados manualmente.

In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Carregue o [modelo FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) pré-treinado e seu tokenizer diretamente do HuggingFace. Observe que você usará a [versão pequena](https://huggingface.co/google/flan-t5-base) do FLAN-T5. A configuração `torch_dtype=torch.bfloat16` especifica o tipo de memória a ser usado por este modelo.

In [4]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

É possível extrair a quantidade de parâmetros do modelo e descobrir quantos deles são treináveis. A função a seguir pode ser usada para fazer isso, nesta fase você não precisa entrar em detalhes.

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


<a nome='1.3'></a>
### 1.3 - Teste o modelo com inferência de disparo zero

Teste o modelo com a inferência de disparo zero. Você pode ver que o modelo tem dificuldade para resumir o diálogo em comparação com o resumo da linha de base, mas extrai algumas informações importantes do texto que indicam que o modelo pode ser ajustado para a tarefa em questão.

In [6]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

<a nome='2'></a>
## 2 - Execute o ajuste fino completo

<a nome='2.1'></a>
### 2.1 - Pré-processar o conjunto de dados de resumo de diálogo

Você precisa converter os pares diálogo-resumo (prompt-resposta) em instruções explícitas para o LLM. Anexe uma instrução ao início da caixa de diálogo com `Resumir a seguinte conversa` e ao início do resumo com `Resumo` da seguinte forma:

Prompt de treinamento (diálogo):
```
Resuma a seguinte conversa.

     Chris: Esta é a parte dele na conversa.
     Antje: Esta é a parte dela na conversa.
    
Resumo:
```

Resposta do treinamento (resumo):
```
Tanto Chris quanto Antje participaram da conversa.
```

Em seguida, pré-processe o conjunto de dados de resposta imediata em tokens e extraia seus `input_ids` (1 por token).

In [7]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Para economizar algum tempo no laboratório, você fará uma subamostra do conjunto de dados:

In [8]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Verifique as formas de todas as três partes do conjunto de dados:

In [9]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


O conjunto de dados de saída está pronto para ajuste fino.

<a nome='2.2'></a>
### 2.2 - Ajustar o modelo com o conjunto de dados pré-processado

Agora utilize a classe Hugging Face `Trainer` integrada (veja a documentação [aqui](https://huggingface.co/docs/transformers/main_classes/trainer)). Passe o conjunto de dados pré-processado com referência ao modelo original. Outros parâmetros de treinamento são encontrados experimentalmente e não há necessidade de entrar em detalhes sobre eles no momento.

In [10]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
trainer.train()

Step,Training Loss
1,49.000000


TrainOutput(global_step=1, training_loss=49.0, metrics={'train_runtime': 4.2164, 'train_samples_per_second': 1.897, 'train_steps_per_second': 0.237, 'total_flos': 5478058819584.0, 'train_loss': 49.0, 'epoch': 0.06})

Treinar uma versão totalmente ajustada do modelo levaria algumas horas em uma GPU. Para economizar tempo, baixe um ponto de verificação do modelo totalmente ajustado para usar no restante deste notebook. Esse modelo totalmente ajustado também será chamado de **modelo de instrução** neste laboratório.

O tamanho do modelo de instrução baixado é de aproximadamente 1 GB.

Crie uma instância da classe `AutoModelForSeq2SeqLM` para o modelo de instrução:

In [12]:
instruct_model_name = 'truocpham/flan-dialogue-summary-checkpoint'

instruct_model = AutoModelForSeq2SeqLM.from_pretrained( instruct_model_name, torch_dtype=torch.bfloat16)

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

<a nome='2.3'></a>
### 2.3 - Avaliar o Modelo Qualitativamente (Avaliação Humana)

Tal como acontece com muitos aplicativos GenAI, uma abordagem qualitativa em que você se pergunta "Meu modelo está se comportando da maneira que deveria?" geralmente é um bom ponto de partida. No exemplo abaixo (o mesmo com o qual iniciamos este caderno), você pode ver como o modelo ajustado é capaz de criar um resumo razoável do diálogo em comparação com a incapacidade original de entender o que está sendo pedido ao modelo.

In [13]:
# Verificar se a GPU está disponível e mover tensores, se necessário
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Mover modelos para o mesmo dispositivo
original_model = original_model.to(device)
instruct_model = instruct_model.to(device)

index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
##Person1: Have you considered upgrading your system? ##Person2: Yes, but you might want to add a painting program to your software. ##Person1: I'd like to make up my own flyers and banners. ##Person2: I'd like to make up my own flyers and banners. ##Person1: I'd like to make up my own flyers and banners. ##Person2: I'd like to make up my own flyers and banners. ##Person1: I'd like to make up my own flyers and banners. ##Person2: I'd like to make up my own flyers and banners. ##Person1: I'd like to make up my own flyers and banners. ##Person2: I'd like to make up my own flyers and
------------------------------------------------------------------------------------

<a nome='2.4'></a>
### 2.4 - Avalie o modelo quantitativamente (com métrica ROUGE)

A [métrica ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) ajuda a quantificar a validade dos resumos produzidos pelos modelos. Ele compara os resumos a um resumo de "linha de base" que geralmente é criado por um ser humano. Embora não seja perfeito, indica o aumento geral na eficácia da sumarização que conseguimos através do ajuste fino.

In [14]:
rouge = evaluate.load('rouge')

Gere as saídas para a amostra do conjunto de dados de teste (apenas 10 diálogos e resumos para economizar tempo) e salve os resultados.

In [15]:
# Verificar se a GPU está disponível e mover tensores, se necessário
device = 'cuda' if torch.cuda.is_available() else 'cpu'

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)  # Definindo o dispositivo aqui

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns=['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df


,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,The memo is to go out as a memo to all employe...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,Employees who use Instant Messages will receiv...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,Memo - Employees are required to use instant m...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,People are complaining about the traffic jam n...,#Person2# got stuck in traffic again. #Person1...
4,#Person2# decides to follow #Person1#'s sugges...,The car was a terrible choice for Person1 beca...,#Person2# got stuck in traffic again. #Person1...
5,#Person2# complains to #Person1# about the tra...,The driver of the car is a liar.,#Person2# got stuck in traffic again. #Person1...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...
7,#Person1# tells Kate that Masha and Hero are g...,#Person1: Masha and Hero are getting divorced....,Masha and Hero are getting divorced. Kate can'...
8,#Person1# and Kate talk about the divorce betw...,#Person1#: I'm so surprised that Masha and Her...,Masha and Hero are getting divorced. Kate can'...
9,#Person1# and Brian are at the birthday party ...,Brian has a birthday party.,Brian's birthday is coming. #Person1# invites ...


Avalie os modelos que calculam as métricas ROUGE. Observe a melhora nos resultados!

In [16]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.24671640777523127, 'rouge2': 0.08271290089838478, 'rougeL': 0.20464767203601503, 'rougeLsum': 0.2067127953185296}
INSTRUCT MODEL:
{'rouge1': 0.41026607717457186, 'rouge2': 0.17840645241958838, 'rougeL': 0.2977022096267017, 'rougeLsum': 0.2987374187518165}


O arquivo `data/dialogue-summary-training-results.csv` contém uma lista pré-preenchida de todos os resultados do modelo que você pode usar para avaliar uma seção maior de dados. Vamos fazer isso para cada um dos modelos:

In [17]:
import requests

url = "https://raw.githubusercontent.com/Ryota-Kawamura/Generative-AI-with-LLMs/main/Week-2/data/dialogue-summary-training-results.csv"
response = requests.get(url)

if response.status_code == 200:
    with open("dialogue-summary-training-results.csv", "wb") as f:
        f.write(response.content)
    print("Arquivo baixado com sucesso!")
else:
    print("Erro ao baixar o arquivo.")


Arquivo baixado com sucesso!


In [18]:
results = pd.read_csv("/content/dialogue-summary-training-results.csv")

human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}
INSTRUCT MODEL:
{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}


The results show substantial improvement in all ROUGE metrics:

In [20]:
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL
rouge1: 18.82%
rouge2: 10.43%
rougeL: 13.70%
rougeLsum: 13.69%


<a nome='3'></a>
## 3 - Execute o ajuste fino eficiente de parâmetros (PEFT)

Agora, vamos realizar o **Ajuste Fino Eficiente de Parâmetros (PEFT)** em vez do "ajuste fino completo" como você fez acima. PEFT é uma forma de ajuste fino de instrução que é muito mais eficiente do que o ajuste fino completo - com resultados de avaliação comparáveis, como você verá em breve.

PEFT é um termo genérico que inclui **Adaptação de baixo nível (LoRA)** e ajuste imediato (que NÃO É O MESMO que engenharia imediata!). Na maioria dos casos, quando alguém diz PEFT, normalmente quer dizer LoRA. LoRA, em um nível muito alto, permite ao usuário ajustar seu modelo usando menos recursos computacionais (em alguns casos, uma única GPU). Após o ajuste fino para uma tarefa, caso de uso ou locatário específico com LoRA, o resultado é que o LLM original permanece inalterado e surge um “adaptador LoRA” recém-treinado. Este adaptador LoRA é muito, muito menor que o LLM original - da ordem de uma% de um dígito do tamanho original do LLM (MBs vs GBs).

Dito isto, no momento da inferência, o adaptador LoRA precisa ser reunido e combinado com seu LLM original para atender à solicitação de inferência. A vantagem, entretanto, é que muitos adaptadores LoRA podem reutilizar o LLM original, o que reduz os requisitos gerais de memória ao atender a múltiplas tarefas e casos de uso.

<a nome='3.1'></a>
### 3.1 - Configure o modelo PEFT/LoRA para ajuste fino

Você precisa configurar o modelo PEFT/LoRA para ajuste fino com um novo adaptador de camada/parâmetro. Usando PEFT/LoRA, você congela o LLM subjacente e treina apenas o adaptador. Dê uma olhada na configuração LoRA abaixo. Observe o hiperparâmetro rank (`r`), que define o rank/dimensão do adaptador a ser treinado.

In [22]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Adicione camadas/parâmetros do adaptador LoRA ao LLM original a ser treinado.

In [23]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


<a nome='3.2'></a>
### 3.2 - Adaptador PEFT de trem

Defina argumentos de treinamento e crie uma instância `Trainer`.

In [24]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [25]:
peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
1,49.250000


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/spiece.model',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

Esse treinamento foi realizado em um subconjunto de dados. Para carregar um modelo PEFT totalmente treinado, leia um ponto de verificação de um modelo PEFT do S3.

Verifique se o tamanho deste modelo é bem menor que o LLM original:

Prepare este modelo adicionando um adaptador ao modelo FLAN-T5 original. Você está definindo `is_trainable=False` porque o plano é apenas realizar inferência com este modelo PEFT. Se você estivesse preparando o modelo para treinamento adicional, você definiria `is_trainable=True`.

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-dialogue-summary-checkpoint-from-s3/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [26]:
from peft import PeftModel, PeftConfig

# huggin face alternative
peft_dialogue_summary_checkpoint = 'intotheverse/peft-dialogue-summary-checkpoint'

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       peft_dialogue_summary_checkpoint, #'./peft-dialogue-summary-checkpoint-from-s3/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)


adapter_config.json:   0%|          | 0.00/334 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

O número de parâmetros treináveis será `0` devido à configuração `is_trainable=False`:

In [27]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


<a nome='3.3'></a>
### 3.3 - Avaliar o Modelo Qualitativamente (Avaliação Humana)

Faça inferências para o mesmo exemplo das seções [1.3](#1.3) e [2.3](#2.3), com o modelo original, totalmente ajustado e modelo PEFT.

In [29]:
# Verificar se a GPU está disponível
device = 'cuda' if torch.cuda.is_available() else 'cpu'

index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

# Converter o prompt para o dispositivo correto
prompt_tensor = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Converter todos os modelos para o dispositivo correto
original_model = original_model.to(device)
instruct_model = instruct_model.to(device)
peft_model = peft_model.to(device)

# Gerar saídas usando os modelos
original_model_outputs = original_model.generate(input_ids=prompt_tensor, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=prompt_tensor, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=prompt_tensor, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

# Exibir resultados
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Is there anything you need?
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1# suggests #Person2# upgrading #Person2#'s system, hardware, and CD-ROM drive. #Person2# thinks it's great.
---------------------------------------------------------------------------------------------------
PEFT MODEL: #Person1# recommends adding a painting program to #Person2#'s software and upgrading hardware. #Person2# also wants to upgrade the hardware because it's outdated now.


<a nome='3.4'></a>
### 3.4 - Avalie o modelo quantitativamente (com métrica ROUGE)
Realize inferências para a amostra do conjunto de dados de teste (apenas 10 diálogos e resumos para economizar tempo).

In [31]:
import torch

# Verificar se a GPU está disponível
device = 'cuda' if torch.cuda.is_available() else 'cpu'

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    human_baseline_text_output = human_baseline_summaries[idx]

    # Mover os modelos para o mesmo dispositivo
    original_model = original_model.to(device)
    instruct_model = instruct_model.to(device)
    peft_model = peft_model.to(device)

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df


,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,Employees are required to take a memo out to t...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,#Person1: This memo is to be distributed to al...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,Employees who use instant messaging will be ba...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic is bad.,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and #Person1# s...
4,#Person2# decides to follow #Person1#'s sugges...,Person1: I'm finally here.,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and #Person1# s...
5,#Person2# complains to #Person1# about the tra...,#Person1#: I got stuck in traffic again. #Pers...,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and #Person1# s...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
8,#Person1# and Kate talk about the divorce betw...,The conversation is going on.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
9,#Person1# and Brian are at the birthday party ...,Brian's birthday is coming up.,Brian's birthday is coming. #Person1# invites ...,Brian remembers his birthday and invites #Pers...


In [32]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2667554929387822, 'rouge2': 0.08369379412857675, 'rougeL': 0.22528238760486846, 'rougeLsum': 0.22666055282311803}
INSTRUCT MODEL:
{'rouge1': 0.41026607717457186, 'rouge2': 0.17840645241958838, 'rougeL': 0.2977022096267017, 'rougeLsum': 0.2987374187518165}
PEFT MODEL:
{'rouge1': 0.3725351062275605, 'rouge2': 0.12138811933618107, 'rougeL': 0.27620639623170606, 'rougeLsum': 0.2758134870822362}


Notice, that PEFT model results are not too bad, while the training process was much easier!

Você já calculou a pontuação ROUGE no conjunto de dados completo, depois de carregar os resultados do arquivo `data/dialogue-summary-training-results.csv`. Carregue agora os valores do modelo PEFT e verifique seu desempenho em comparação com outros modelos.

In [33]:
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values
peft_model_summaries     = results['peft_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}
INSTRUCT MODEL:
{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}
PEFT MODEL:
{'rouge1': 0.40810631575616746, 'rouge2': 0.1633255794568712, 'rougeL': 0.32507074586565354, 'rougeLsum': 0.3248950182867091}


Os resultados mostram menos melhorias em relação ao ajuste fino completo, mas os benefícios do PEFT normalmente superam as métricas de desempenho ligeiramente inferiores.

Calcule a melhoria do PEFT em relação ao modelo original:

In [34]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 17.47%
rouge2: 8.73%
rougeL: 12.36%
rougeLsum: 12.34%


Agora calcule a melhoria do PEFT em relação a um modelo totalmente ajustado:

In [35]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: -1.35%
rouge2: -1.70%
rougeL: -1.34%
rougeLsum: -1.35%


Aqui você vê uma pequena diminuição percentual nas métricas ROUGE em comparação com o ajuste fino completo. No entanto, o treinamento requer muito menos recursos de computação e memória (geralmente apenas uma GPU).